In [11]:
# Import neccesary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.models import Model
from keras.layers import Flatten, Dense

from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

In [12]:
# Train the model

model = MobileNet(input_shape=(224,224,3), include_top=False)

for layer in model.layers:
    layer.trainable = False

X = Flatten()(model.output)
X = Dense(units= 3, activation='softmax')(X)

model = Model(model.input, X)
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32)      128 

In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
# Data Preprocessing

# Preprocessing Train Data

train_datagen = ImageDataGenerator(
    zoom_range = 0.2,
    shear_range= 0.2,
    horizontal_flip=True,
    rescale=1./255
)

train_data = train_datagen.flow_from_directory(
    directory='../vid_to_img/',
    target_size=(224,224),
    batch_size=32
)

print(train_data.class_indices)


# Preprocessing Validation Data

val_datagen = ImageDataGenerator(rescale=1./255)

val_data = val_datagen.flow_from_directory(
    directory='../vid_to_img/',
    target_size=(224,224),
    batch_size=32
)

print(val_data.class_indices)

Found 4746 images belonging to 3 classes.
{'angry': 0, 'happy': 1, 'sad': 2}
Found 4746 images belonging to 3 classes.
{'angry': 0, 'happy': 1, 'sad': 2}


In [6]:
val_datagen = ImageDataGenerator(rescale=1./255)

val_data = val_datagen.flow_from_directory(
    directory='../vid_to_img/',
    target_size=(224,224),
    batch_size=32
)

val_data.class_indices


Found 4746 images belonging to 3 classes.


{'angry': 0, 'happy': 1, 'sad': 2}

In [ ]:
# Vizualization

t_img , label = train_data.next()

def plotImages(img_arr, label):
  """
  input  :- images array 
  output :- plots the images 
  """
  count = 0
  for im, l in zip(img_arr,label) :
    plt.imshow(im)
    plt.title(im.shape)
    plt.axis = False
    plt.show()
    
    count += 1
    if count == 10:
      break

plotImages(t_img, label)
     

In [9]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# early stopping
es = EarlyStopping(monitor='val_accuracy', min_delta= 0.01 , patience= 5, verbose= 1, mode='auto')

# model check point
mc = ModelCheckpoint(filepath="eunah_model.h5", monitor= 'val_accuracy', verbose= 1, save_best_only= True, mode = 'auto')

# puting call back in a list 
call_back = [es, mc]

In [10]:
hist = model.fit_generator(train_data, 
                           steps_per_epoch= 10, 
                           epochs= 30, 
                           validation_data= val_data, 
                           validation_steps= 8, 
                           callbacks=[es,mc])

C:\Users\James\AppData\Local\Temp\ipykernel_9712\564963302.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(train_data,


Epoch 1/30
10/10 [==============================] - ETA: 0s - loss: 7.3728 - accuracy: 0.5250
Epoch 1: val_accuracy improved from -inf to 0.87109, saving model to eunah_model.h5
10/10 [==============================] - 11s 995ms/step - loss: 7.3728 - accuracy: 0.5250 - val_loss: 0.9725 - val_accuracy: 0.8711
Epoch 2/30


c:\Users\James\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


10/10 [==============================] - ETA: 0s - loss: 0.6142 - accuracy: 0.8750
Epoch 2: val_accuracy improved from 0.87109 to 0.99219, saving model to eunah_model.h5
10/10 [==============================] - 9s 883ms/step - loss: 0.6142 - accuracy: 0.8750 - val_loss: 0.0629 - val_accuracy: 0.9922
Epoch 3/30
10/10 [==============================] - ETA: 0s - loss: 0.1242 - accuracy: 0.9750
Epoch 3: val_accuracy did not improve from 0.99219
10/10 [==============================] - 8s 821ms/step - loss: 0.1242 - accuracy: 0.9750 - val_loss: 0.0446 - val_accuracy: 0.9922
Epoch 4/30
10/10 [==============================] - ETA: 0s - loss: 0.0747 - accuracy: 0.9875
Epoch 4: val_accuracy improved from 0.99219 to 0.99609, saving model to eunah_model.h5
10/10 [==============================] - 8s 792ms/step - loss: 0.0747 - accuracy: 0.9875 - val_loss: 0.0053 - val_accuracy: 0.9961
Epoch 5/30
10/10 [==============================] - ETA: 0s - loss: 0.0117 - accuracy: 0.9969  
Epoch 5: val_ac